In [ ]:
import tensorflow as tf
import os

import cv2
import imghdr

#removendo fotos com extensões não compativeis
data_dir = 'train'

image_exts = ['jpeg', 'jpg', 'bmp', 'png']

for image_class in os.listdir(data_dir):#percorrendo todas as pastas 
  if os.path.isdir(image_class):
   for image in os.listdir(os.path.join(data_dir, image_class)): #percorrendo todas as fotos de todas as pastas
     image_path = os.path.join(data_dir,image_class,image)
    
     try:
       img = cv2.imread(image_path)
       tip = imghdr.what(image_path)
       if tip not in image_exts:
         print("Imagem não está na extensao certa{}".format(image_path))
         os.remove(image_path)
     except Exception as e:
       print("Problema com a imagem{}".format(image_path))    

import numpy as np
from matplotlib import pyplot as plt

 #Load dataset
 data = tf.keras.utils.image_dataset_from_directory('train')
 data_iterator = data.as_numpy_iterator()
 batch = data_iterator.next()
 batch[0].shape
 batch[1]

fig, ax = plt.subplots(ncols=5, figsize=(20,20)) #checando numero das categorias com as imagens
for idx, img in enumerate(batch[0][:5]):
  ax[idx].imshow(img.astype(int))
  ax[idx].title.set_text(batch[1][idx])
    
#Processando os dados
#escalonando as imagens

data= data.map(lambda x, y: (x/255, y)) #para o valor do rgb ficar entre 0 e 1
scaled_iterator = data.as_numpy_iterator()
batch= scaled_iterator.next() 
batch[0].max()

#Dividindo entre treino, validacao e teste

train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)
 
from logging import logProcesses
#Construindo a deep learning
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

model = Sequential()

model.add(Conv2D(16,(3,3),1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32,(3,3),1, activation='relu'))
model.add(MaxPooling2D())


model.add(Conv2D(16,(3,3),1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
#model.add(Dense(1, activatio='sigmoid'))

model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])
model.summary()

#Treino

logdir= 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])   

fig = plt.figure()
plt.plot(hist.history['loss'], color='blue', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plot.show()
    

    